In [2]:
import pandas as pd
import os
import openpyxl
import geopandas as gpd
import gc
gc.collect()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
match = pd.read_excel('../Resources/COPHistoricAttendance.xlsx', sheet_name='SHP_MATCH')[['OrigName','Country']]
countries = gpd.read_file('../Resources/countries').merge(
    match,
    left_on='name',right_on='OrigName',how='inner'
).drop(['name','OrigName'],axis=1)


In [ ]:
attendance = pd.read_excel('../Resources/COPHistoricAttendance.xlsx', sheet_name='Attendance')

In [ ]:
attendance

In [1]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar,LTLine,LAParams, LTTextLineHorizontal
import os


In [59]:
dfli = []

In [91]:
COP = 'COP27'

path = f'../Resources/AttendancePDFs/{COP}_Attendance.pdf'
page = 0
datalist = []
category = ''
country = ''
text = ''

fonts = {
    'COP23': {
        'category':{'font_name':'BCDEEE+PalatinoLinotype-Bold','font_size':12},
        'country':{'font_name':'BCDFEE+PalatinoLinotype-BoldItalic','font_size':9},
        'text':{'font_name':'BCDGEE+PalatinoLinotype-Roman','font_size':9},
    },
    'COP24': {
        'category':{'font_name':'BCDEEE+Cambria-Bold','font_size':12},
        'country':{'font_name':'BCDFEE+Cambria-BoldItalic','font_size':9},
        'text':{'font_name':'BCDGEE+Cambria','font_size':9},
    },
    'COP25': {
        'category':{'font_name':'BCDEEE+Cambria-Bold','font_size':12},
        'country':{'font_name':'BCDFEE+Cambria-BoldItalic','font_size':9},
        'text':{'font_name':'BCDGEE+Cambria','font_size':9},
    },
    'COP26': {
        'category':{'font_name':'TimesNewRomanPS-BoldMT','font_size':14},
        'country':{'font_name':'TimesNewRomanPS-BoldMT','font_size':12},
        'text':{'font_name':'TimesNewRomanPSMT','font_size':9},
    },
    'COP27': {
        'category':{'font_name':'TimesNewRomanPS-BoldMT','font_size':14},
        'country':{'font_name':'TimesNewRomanPS-BoldMT','font_size':12},
        'text':{'font_name':'TimesNewRomanPSMT','font_size':9},
    }
}

for page_layout in extract_pages(
    path, 
    laparams = LAParams(

    )
):

    for element in page_layout:
        if isinstance(element, LTTextContainer):
            textitems = []
            for text_line in element:
                if isinstance(text_line, LTTextLineHorizontal):
                    for character in text_line:
                        if isinstance(character, LTChar):
                            fontsize = int(character.size)
                            fontname = character.fontname
                            break
                        else:\
                            print('XXXX')

                    if fontname == fonts[COP]['category']['font_name'] and fontsize == fonts[COP]['category']['font_size']:
                        category = text_line.get_text().strip()
                    if fontname == fonts[COP]['country']['font_name'] and fontsize == fonts[COP]['country']['font_size']:
                        country = text_line.get_text().strip()
                    if fontname == fonts[COP]['text']['font_name'] and fontsize == fonts[COP]['text']['font_size']:
                        textitems.append(text_line.get_text().strip())
                    
            if category != '' and country != '' and len(textitems) != 0:
                item = [category,country,textitems]
#                 print(item)
                datalist.append(item)
            #GUIDE
#             print([fontname,fontsize,element.get_text()])

    page +=1
                

In [92]:
df = pd.DataFrame(datalist)
df.columns = ['category','country','delegate']
df['COP'] = COP
df

,category,country,delegate,COP
0,Parties,Albania,"[H.E. Mr. Edi Rama, Prime Minister of the Repu...",COP27
1,Parties,Albania,"[H.E. Ms. Mirela Furxhi, Minister, Ministry of...",COP27
2,Parties,Albania,"[H.E. Ms. Frida Krifca, Minister of Agricultur...",COP27
3,Parties,Albania,"[Mr. Genti Haderaj, Director, Directorate of P...",COP27
4,Parties,Albania,"[Ms. Eneida Rabdishta, Expert, Directorate of ...",COP27
...,...,...,...,...
30350,Non-governmental organizations,ZOI Environment Network,"[Ms. Ellie Kate Kinney, Associate, Conflict Ob...",COP27
30351,Non-governmental organizations,Zoological Society of London,[Ms. Bethan Laughlin],COP27
30352,Non-governmental organizations,Zoological Society of London,"[Ms. Nathalie Pettorelli, Senior Research Fell...",COP27
30353,Non-governmental organizations,Zurich University of Applied Sciences,[Ms. Raphaela Andrea Kotsch],COP27


In [94]:
frame= pd.concat(dfli,axis=0,ignore_index=True)

In [98]:
frame.to_excel('HistoricalAttendanceListPDF.xlsx')

In [29]:
# Open the Excel workbook
excel_file_path = '../Resources/COP28AttendanceList.xlsx'
workbook = openpyxl.load_workbook(excel_file_path)

# Get the sheet names
sheet_names = workbook.sheetnames

li =[]

for sheet in sheet_names:
    if sheet not in ['SKIP','CORRESP']:
        df = pd.read_excel(excel_file_path,sheet_name=sheet)
        df['category'] = sheet
        li.append(df)
        
frame = pd.concat(li,axis=0,ignore_index=True).rename(columns={'nominator':'country'})

In [39]:
frame['COP'] = 'COP28'

In [25]:
frame2 = pd.read_excel('HistoricalAttendanceListPDF.xlsx')
frame2['delegate'] = frame2['delegate'].apply(eval)
frame2['name'] = frame2['delegate'].apply(lambda x: x[0])
frame2['func_title'] = frame2['delegate'].apply(lambda x: '/n'.join(x[1:]))

frame2.category = frame2.category.replace(
    {
        'Specialized agencies and related organizations':'Spec. agencies and related orgs',
        'United Nations Secretariat units and bodies':'United Nations Secretariat unit',
        'United Nations secretariat units and bodies':'United Nations Secretariat unit'
    }
)

In [41]:
frame2 = frame2[['country','name','func_title','category','COP']]

In [42]:
frame = pd.concat([frame, frame2],axis=0,ignore_index=True)

In [44]:
frame.to_excel('HistoricalAttendanceListPDF.xlsx',index=False)